#### check model: https://huggingface.co/jackhhao/jailbreak-classifier

In [1]:
from transformers import pipeline
from datasets import load_dataset
import evaluate


/opt/anaconda3/envs/qualifier-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipe = pipeline("text-classification", model="jackhhao/jailbreak-classifier")
dataset = load_dataset("jackhhao/jailbreak-classification")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'type'],
        num_rows: 1044
    })
    test: Dataset({
        features: ['prompt', 'type'],
        num_rows: 262
    })
})

In [6]:
test_dataset = dataset["test"]

In [4]:
# Load all metrics from the evaluate library
f1_metric = evaluate.load("f1")
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

In [13]:
#y_test if benign -> 0 else 1

y_test = [0 if x == "benign" else 1 for x in test_dataset["type"]]

In [17]:
def calculate_preds(model_pipeline, dataset):
    predictions = []
    references = []
    i = 0 
    for sample in dataset:
        print(i)
        i+=1
        # Get model prediction with truncation
        prediction = model_pipeline(sample["prompt"], truncation=True)[0]
        
        # Map text labels to numerical labels
        if prediction["label"].lower() == "benign":
            predicted_label = 0
        elif prediction["label"].lower() == "jailbreak":
            predicted_label = 1
        else:
            raise ValueError(f"Unexpected label format: {prediction['label']}")

        predictions.append(predicted_label)
        references.append(1 if sample["type"].lower() == "jailbreak" else 0)  # Adjust label mapping accordingly
    return predictions, references    

In [18]:
# Define a function to evaluate predictions on the test set
def evaluate_model(model_pipeline, dataset):
    predictions, references = calculate_preds(model_pipeline, dataset)

    # Compute each metric
    f1_score = f1_metric.compute(predictions=predictions, references=references, average="binary")
    accuracy = accuracy_metric.compute(predictions=predictions, references=references)
    recall = recall_metric.compute(predictions=predictions, references=references, average="binary")
    precision = precision_metric.compute(predictions=predictions, references=references, average="binary")

    return {
        "F1 Score": f1_score["f1"],
        "Accuracy": accuracy["accuracy"],
        "Recall": recall["recall"],
        "Precision": precision["precision"]
    }, predictions, references

# Evaluate on the test set
metrics, predictions, references = evaluate_model(pipe, test_dataset)
print("Evaluation Results:")
for metric_name, metric_value in metrics.items():
    print(f"{metric_name}: {metric_value}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
Evaluation Results:
F1 Score: 0.9747292418772563
Accuracy: 0.9

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()



# Plot the confusion matrix
plot_confusion_matrix(y_test, y_pred_cpu, labels=['benign', 'jailbreak'])